参考にさせていただいたサイト。ほぼそのままです。

https://note.com/npaka/n/n8a435f0c8f69

# GPT2でファインチューニングしたモデルから文章生成する。

公開されているRinnaの学習モデルをファインチューニングしたモデルから文章の生成を行う。

環境

 - Clab PRO
 - Google Drive

ディレクトリ

./output : 学習したモデルを保存

In [ ]:
# Google Driveに接続
from google.colab import drive 
drive.mount('/content/drive')
!mkdir -p '/content/drive/My Drive/work/'
%cd '/content/drive/My Drive/work/'

Mounted at /content/drive
/content/drive/My Drive/work


In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 4.1 MB/s 
     |████████████████████████████████| 77 kB 5.7 MB/s 
     |████████████████████████████████| 880 kB 59.8 MB/s 
     |████████████████████████████████| 596 kB 42.4 MB/s 
     |████████████████████████████████| 6.6 MB 41.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=9a6e11f7cb52bd86a25861b24245d2e08cc3ebdc5aebe72083135f087a47d4d5
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Runtime 再起動

In [ ]:
# 作業フォルダへ
%cd '/content/drive/My Drive/work/'

/content/drive/My Drive/work


In [ ]:
# Huggingface Datasetsのインストール
!pip install datasets==1.2.1
# Sentencepieceのインストール
!pip install sentencepiece==0.1.91

     |████████████████████████████████| 159 kB 4.2 MB/s 
     |████████████████████████████████| 69 kB 6.1 MB/s 
     |████████████████████████████████| 212 kB 51.8 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.0
    Uninstalling tqdm-4.64.0:
      Successfully uninstalled tqdm-4.64.0
     |████████████████████████████████| 1.1 MB 3.9 MB/s 


「./transformers/examples/language-modeling/run_clm.py」の編集。
「rinna」の日本語GPT-2モデルは「AutoTokenizer」ではなく「T5Tokenizer」を使う必要があります。

In [ ]:
from transformers import T5Tokenizer, AutoModelForCausalLM
import re

# トークナイザーとモデルの準備
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-small")
model = AutoModelForCausalLM.from_pretrained("output/")

# テスト生成
input = tokenizer.encode(
    """<s>アムロ：こんにちは</s>
    <s>シャア：ララァはどこに行った？</s>""", return_tensors="pt")
output = model.generate(input,
                        do_sample=True,
                        top_p=0.90,
                        top_k=0,
                        max_length=200,
                        skip_special_tokens=True,
                        pad_token_id=tokenizer.pad_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        bad_word_ids=[[tokenizer.unk_token_id]],
                        num_return_sequences=10
                        )

# 結果表示
for list_output in output:
  print(re.sub(r'<s>|<unk>|\[PAD\]|\[SEP\]| ', '',
               tokenizer.decode(list_output).replace('</s>', '\n')))


/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:195: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


アムロ:こんにちは
シャア:ララァはどこに行った?
ハサウェイダミー

アムロ:こんにちは
シャア:ララァはどこに行った?
アムロえず、落ちないはずだ

アムロ:こんにちは
シャア:ララァはどこに行った?
アムロなぜ、シャアは地球を寒冷化するだけのエネルギーを使わないのか?

アムロ:こんにちは
シャア:ララァはどこに行った?
アムロレーガン

アムロ:こんにちは
シャア:ララァはどこに行った?
アムロそこかっ

アムロ:こんにちは
シャア:ララァはどこに行った?
アムロ:ああ、地球連邦政府はジオンの残党狩りをしているのか

アムロ:こんにちは
シャア:ララァはどこに行った?
シャア:"...えっ?

アムロ:こんにちは
シャア:ララァはどこに行った?
アムロ:そこか?

アムロ:こんにちは
シャア:ララァはどこに行った?
アムロサラダを一緒に食べるのは?

アムロ:こんにちは
シャア:ララァはどこに行った?
アムロ:そこか?

